In [ ]:
# Ejemplo de como armar un chat con message history.
# No estoy usando langgraph.
# https://python.langchain.com/docs/how_to/chatbots_memory/

In [1]:
%%capture outputtext

# Colab:
!pip install langchain-community
!pip install langchain-ollama
# !pip install langgraph
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve &>/dev/null&

# # Local:
# import os
# environment= 'local'
# os.system('ollama serve &>/dev/null&')

In [2]:
# !ollama pull llama3.1
# !ollama pull llama3.2-vision
!ollama pull qwen2.5vl:7b

In [5]:
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

model = ChatOllama(
    model = "qwen2.5vl:7b",
    # model = "llama3.2-vision",
    # model = "llama3.1",
    # model = "llama3.1:8b",
    # temperature = 0.8,  # default en llama
)


In [10]:
# Ejemplo muy básico.
# Agrego la respuesta del modelo al diccionario.

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg.content)

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
            AIMessage(content=ai_msg.content),
            HumanMessage(
                content="Now translate it to Spanish."
            ),
        ],
    }
)
print(ai_msg.content)

I just said "J'adore la programmation." which translates from English to "I love programming."
Me encanta la programación.
